# 轮廓系数 (Silhouette Score)

**核心概念**: 轮廓系数是评估聚类质量的内部指标，无需真实标签即可衡量聚类的紧凑性和分离性

## 数学定义

对于样本 $i$，其轮廓系数定义为:

$$s(i) = \frac{b(i) - a(i)}{\max\{a(i), b(i)\}}$$

其中:
- $a(i)$: **簇内距离** - 样本 $i$ 到同簇其他样本的平均距离 (紧凑性)
- $b(i)$: **最近簇距离** - 样本 $i$ 到最近邻簇所有样本的平均距离 (分离性)

## 取值范围与含义

| 取值范围 | 含义 |
|---------|------|
| $s(i) \approx 1$ | 样本被正确分配，远离邻近簇 |
| $s(i) \approx 0$ | 样本位于两个簇的边界 |
| $s(i) < 0$ | 样本可能被错误分配 |

整体轮廓系数为所有样本轮廓系数的平均值。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.datasets import load_iris, make_blobs
from sklearn.preprocessing import StandardScaler

# 设置随机种子
np.random.seed(42)

# 配置 matplotlib
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 数据准备

In [ ]:
# 加载鸢尾花数据集
iris = load_iris()
X = iris.data
y_true = iris.target

print(f"数据集形状: {X.shape}")
print(f"真实类别数: {len(np.unique(y_true))}")

## 2. 使用轮廓系数确定最佳 K 值

与肘部法则相比，轮廓系数提供了更客观的聚类质量评估，可以直接选择得分最高的 K 值。

In [ ]:
# 计算不同 K 值的轮廓系数
k_range = range(2, 11)  # 轮廓系数要求 K >= 2
silhouette_scores = []
inertias = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = kmeans.fit_predict(X)
    
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    inertias.append(kmeans.inertia_)
    
    print(f"K={k}: 轮廓系数={score:.4f}, 惯性={kmeans.inertia_:.2f}")

# 找到最佳 K
best_k = k_range[np.argmax(silhouette_scores)]
best_score = max(silhouette_scores)
print(f"\n最佳 K 值: {best_k} (轮廓系数: {best_score:.4f})")

In [ ]:
# 可视化轮廓系数与肘部法则
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 轮廓系数
axes[0].plot(k_range, silhouette_scores, 'bo-', linewidth=2, markersize=8)
axes[0].axvline(x=best_k, color='r', linestyle='--', 
                label=f'最佳 K={best_k}')
axes[0].scatter([best_k], [best_score], c='red', s=200, zorder=5, marker='*')
axes[0].set_xlabel('聚类数量 K', fontsize=12)
axes[0].set_ylabel('轮廓系数', fontsize=12)
axes[0].set_title('轮廓系数法选择最佳 K', fontsize=14)
axes[0].set_xticks(k_range)
axes[0].set_ylim(0, 1)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 肘部法则对比
axes[1].plot(k_range, inertias, 'go-', linewidth=2, markersize=8)
axes[1].axvline(x=3, color='r', linestyle='--', label='K=3 (肘部)')
axes[1].set_xlabel('聚类数量 K', fontsize=12)
axes[1].set_ylabel('惯性', fontsize=12)
axes[1].set_title('肘部法则', fontsize=14)
axes[1].set_xticks(k_range)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 样本级轮廓系数分析

除了整体轮廓系数，还可以分析每个样本的轮廓系数，识别边界样本和可能被误分的样本。

In [ ]:
def plot_silhouette_analysis(X, n_clusters, ax=None):
    """
    绘制样本级轮廓系数图
    
    参数:
        X: 特征数据
        n_clusters: 簇数量
        ax: matplotlib axes
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    
    # 聚类
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    cluster_labels = kmeans.fit_predict(X)
    
    # 计算轮廓系数
    avg_score = silhouette_score(X, cluster_labels)
    sample_scores = silhouette_samples(X, cluster_labels)
    
    # 绘制每个簇的样本轮廓系数
    y_lower = 10
    
    for i in range(n_clusters):
        # 获取该簇的样本轮廓系数并排序
        cluster_scores = sample_scores[cluster_labels == i]
        cluster_scores.sort()
        
        cluster_size = len(cluster_scores)
        y_upper = y_lower + cluster_size
        
        # 分配颜色
        color = cm.nipy_spectral(float(i) / n_clusters)
        
        # 绘制水平条形图
        ax.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            cluster_scores,
            facecolor=color,
            edgecolor=color,
            alpha=0.7
        )
        
        # 添加簇标签
        ax.text(-0.05, y_lower + 0.5 * cluster_size, str(i), fontsize=10)
        
        y_lower = y_upper + 10
    
    # 添加平均轮廓系数参考线
    ax.axvline(x=avg_score, color='red', linestyle='--', 
               linewidth=2, label=f'平均: {avg_score:.3f}')
    
    ax.set_title(f'K={n_clusters} 的样本轮廓系数分布', fontsize=12)
    ax.set_xlabel('轮廓系数', fontsize=11)
    ax.set_ylabel('簇 (样本按轮廓系数排序)', fontsize=11)
    ax.set_xlim([-0.1, 1.0])
    ax.set_yticks([])
    ax.legend(loc='upper right')
    
    return avg_score

# 绘制 K=3 的轮廓系数分析
fig, ax = plt.subplots(figsize=(10, 6))
plot_silhouette_analysis(X, 3, ax)
plt.tight_layout()
plt.show()

## 4. 不同 K 值的轮廓图对比

通过可视化不同 K 值的轮廓图，可以直观比较聚类效果。

In [ ]:
# 比较不同 K 值
k_values = [2, 3, 4, 5]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for i, k in enumerate(k_values):
    plot_silhouette_analysis(X, k, axes[i])

plt.suptitle('不同 K 值的轮廓系数分布对比', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 轮廓图解读指南

### 良好聚类的特征

1. **宽且均匀的条形**: 各簇样本的轮廓系数较高且一致
2. **超过平均线**: 大部分样本的轮廓系数高于平均值
3. **簇大小均衡**: 各簇包含相近数量的样本

### 问题聚类的特征

1. **细长或参差不齐的条形**: 簇内样本分散
2. **负值样本**: 可能被错误分配的样本
3. **簇大小不均**: 可能存在不自然的分割

In [ ]:
# 使用合成数据演示轮廓分析
# 创建明显分离的簇
X_good, _ = make_blobs(
    n_samples=300, 
    centers=3, 
    cluster_std=0.6, 
    random_state=42
)

# 创建重叠的簇
X_bad, _ = make_blobs(
    n_samples=300, 
    centers=3, 
    cluster_std=2.5, 
    random_state=42
)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 良好分离的数据 - 轮廓分析
plot_silhouette_analysis(X_good, 3, axes[0, 0])
axes[0, 0].set_title('分离良好的数据 - 轮廓分析', fontsize=12)

# 良好分离的数据 - 散点图
kmeans_good = KMeans(n_clusters=3, n_init=10, random_state=42)
labels_good = kmeans_good.fit_predict(X_good)
axes[0, 1].scatter(X_good[:, 0], X_good[:, 1], c=labels_good, cmap='viridis', 
                   s=30, alpha=0.7, edgecolors='k', linewidth=0.5)
axes[0, 1].scatter(kmeans_good.cluster_centers_[:, 0], 
                   kmeans_good.cluster_centers_[:, 1],
                   c='red', marker='X', s=200, edgecolors='k', linewidths=2)
axes[0, 1].set_title(f'分离良好的数据 (轮廓系数: {silhouette_score(X_good, labels_good):.3f})', fontsize=12)

# 重叠的数据 - 轮廓分析
plot_silhouette_analysis(X_bad, 3, axes[1, 0])
axes[1, 0].set_title('重叠的数据 - 轮廓分析', fontsize=12)

# 重叠的数据 - 散点图
kmeans_bad = KMeans(n_clusters=3, n_init=10, random_state=42)
labels_bad = kmeans_bad.fit_predict(X_bad)
axes[1, 1].scatter(X_bad[:, 0], X_bad[:, 1], c=labels_bad, cmap='viridis',
                   s=30, alpha=0.7, edgecolors='k', linewidth=0.5)
axes[1, 1].scatter(kmeans_bad.cluster_centers_[:, 0],
                   kmeans_bad.cluster_centers_[:, 1],
                   c='red', marker='X', s=200, edgecolors='k', linewidths=2)
axes[1, 1].set_title(f'重叠的数据 (轮廓系数: {silhouette_score(X_bad, labels_bad):.3f})', fontsize=12)

plt.tight_layout()
plt.show()

## 6. 错误 K 值的诊断

演示当选择错误的 K 值时，轮廓图如何反映问题。

In [ ]:
# 创建 4 个明显的簇
X_4clusters, y_4clusters = make_blobs(
    n_samples=400,
    centers=4,
    cluster_std=0.7,
    random_state=42
)

# 测试不同的 K 值
k_tests = [2, 3, 4, 5]

fig, axes = plt.subplots(2, 4, figsize=(18, 8))

for i, k in enumerate(k_tests):
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = kmeans.fit_predict(X_4clusters)
    score = silhouette_score(X_4clusters, labels)
    
    # 轮廓分析
    plot_silhouette_analysis(X_4clusters, k, axes[0, i])
    
    # 散点图
    axes[1, i].scatter(X_4clusters[:, 0], X_4clusters[:, 1], 
                       c=labels, cmap='viridis', s=20, alpha=0.7)
    axes[1, i].scatter(kmeans.cluster_centers_[:, 0],
                       kmeans.cluster_centers_[:, 1],
                       c='red', marker='X', s=150, edgecolors='k', linewidths=2)
    axes[1, i].set_title(f'K={k}, 轮廓={score:.3f}')
    
    # 标记真实K值
    if k == 4:
        axes[1, i].set_xlabel('(真实簇数)', fontsize=10, color='red')

plt.suptitle('不同 K 值对真实 4 簇数据的聚类效果 (真实 K=4)', fontsize=14)
plt.tight_layout()
plt.show()

## 7. 计算复杂度说明

轮廓系数的计算需要计算样本对之间的距离，时间复杂度为 $O(n^2)$，空间复杂度同样为 $O(n^2)$。

对于大规模数据集，可以使用采样来近似计算。

In [ ]:
import time

# 测试不同数据规模的计算时间
sample_sizes = [100, 500, 1000, 2000, 5000]
times = []

for n in sample_sizes:
    X_test, _ = make_blobs(n_samples=n, centers=3, random_state=42)
    kmeans = KMeans(n_clusters=3, n_init=3, random_state=42)
    labels = kmeans.fit_predict(X_test)
    
    start = time.time()
    silhouette_score(X_test, labels)
    elapsed = time.time() - start
    times.append(elapsed)
    
    print(f"n={n:5d}: 计算时间 = {elapsed:.4f} 秒")

# 可视化
plt.figure(figsize=(10, 5))
plt.plot(sample_sizes, times, 'bo-', linewidth=2, markersize=8)
plt.xlabel('样本数量', fontsize=12)
plt.ylabel('计算时间 (秒)', fontsize=12)
plt.title('轮廓系数计算时间与样本数量的关系', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 总结

### 轮廓系数的优点

1. **无需真实标签**: 纯内部评估指标
2. **同时考虑紧凑性和分离性**: 综合评估聚类质量
3. **取值范围明确**: [-1, 1]，易于解释
4. **可用于选择最佳 K**: 选择轮廓系数最高的 K 值

### 轮廓系数的局限性

1. **计算复杂度高**: $O(n^2)$，不适合超大数据集
2. **假设凸形簇**: 对非凸形簇评估可能不准确
3. **对密度不均匀敏感**: 不同密度的簇可能影响评估

### 最佳实践

1. 结合肘部法则综合判断最佳 K
2. 查看样本级轮廓系数分布，识别边界样本
3. 对大数据集使用采样近似
4. 轮廓系数 > 0.5 通常表示合理的聚类结构

### 轮廓系数参考标准

| 轮廓系数 | 聚类质量 |
|---------|----------|
| 0.71 - 1.0 | 强结构 |
| 0.51 - 0.70 | 合理结构 |
| 0.26 - 0.50 | 弱结构 |
| < 0.25 | 无实质结构 |